# Web scraping

Cuando necesitamos extraer información publicada en internet, lo ideal es consultar una API, porque:

* Las respuestas contienen información estructurada
* En general, el propio servicio nos da documentación sobre cómo hacer peticiones y qué tipo de información podemos solicitar

Pero muchas veces nos encontramos con información en páginas web (en formato [HTML](https://es.wikipedia.org/wiki/HTML)) que nos gustaría obtener, pero sin API disponible.

Estas páginas `HTML` tienen cierta estructura, aunque con ciertos contras:

* Es más compleja, puede tener muchos niveles de anidamiento
* Es inestable. Están diseñadas para que se vean bien desde el explorador, no para guardar una estructura de consulta. De un día para otro, puede verse alterada por la incorporación de nuevos elementos visuales u otros motivos.
* Puede ser modificada por código cliente (javascript) en diferentes momentos: al cargar la página, al interaccionar con algún elemento, ...

### Ejercicio

Desde tu explorador, consulta el código fuente de una página de tu interés. Por ejemplo, para hacerlo en chrome:

* Accede a la página, p.e. [esta](https://es.wikipedia.org/wiki/HTML).
* Haz click derecho y pulsa `View page source`. Otra opción es pulsar `Inspect`, que además abrirá las herramientas de desarrollador de Chrome, muy útiles para navegar por la estructura de la página.

## Scraping de elementos html

La librería que vamos a utilizar es [Beautiful Soup](https://pypi.org/project/beautifulsoup4/). Nos permite buscar elementos y navegar por la estructura del html fácilmente.

Imaginemos que queremos comparar precios de un determinado modelo de motocicleta de segunda mano. P.e. con [esta búsqueda](https://www.milanuncios.com/motos-de-carretera/duke-390.htm) en milanuncios.

Primero, nos descargamos el html con `requests`.

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm')
page

Podemos ver el contenido examinando la propiedad `content`.

In [ ]:
page.content

Este contenido es solo texto, no tiene estructura. Aún no podemos hacer búsquedas ni navegar por él.

Para hacerlo, creamos una instancia de `Beautiful Soup` y lo parseamos

In [ ]:
soup = BeautifulSoup(page.content, 'html.parser')

Sobre esto, podemos hacer búsquedas con `find` y `find_all` (o `select_one` y `select` si prefieres utilizar [selectores css](https://en.wikipedia.org/wiki/Cascading_Style_Sheets#Selector)). Sobre nuestro ejemplo, vamos a buscar todos los precios. Examinando el código fuente, vemos que son etiquetas `div` con clase `aditem-price`.

In [ ]:
div_precios = soup.find_all('div', class_='aditem-price')
div_precios

`find_all` devuelve una lista de elementos. Sobre ellos, podemos hacer:

`children` para sacar el listado de todos los hijos.

In [ ]:
list(div_precios[0].children)

`get_text()` para sacar el texto de todos los hijos

In [ ]:
div_precios[0].get_text()

Por tanto, para sacar el listado de todos los precios podemos hacer:

In [ ]:
[list(div_precio.children)[0] for div_precio in div_precios]

Tienes más funciones útiles con pequeños ejemplos [aquí](http://akul.me/blog/2016/beautifulsoup-cheatsheet/)

### Ejercicio

Crea un dataframe de pandas en el que cada fila sea un anuncio y tenga como columnas información que consideres relevante: precio, kilómetros, año, cilindrada, texto del anuncio, ...

### Ejercicio

Modifica el código anterior para que, además de bajarse la página actual, navegue por el resto de páginas e incorpore también esos anuncios a tu dataframe.

## Scraping de tablas

A menudo, la información que nos interesa descargar está en tablas y nuestro objetivo es importarlas en tablas de Pandas. Esta conversión suele exigir la manipulación del texto, números y fechas contenidas en la tabla original, lo que nos obligará a repasar cómo realizar esas operaciones y aplicarlas a filas y columnas de las tablas.

La estructura que suelen tener la tablas en `html` es:

```
<table>
    <thead>
        <tr>
            <th>Columna A</th>
            <th>Columna B</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>A1</td>
            <td>B1</td>
        </tr>
        <tr>
            <td>A2</td>
            <td>B2</td>
        </tr>
    </tbody>
</table>   
```

Necesitaremos los siguientes módulos además de `requests` y `BeautifulSoup` importados anteriormente:

In [ ]:
import pandas as pd
import re

Primero, hacemos una petición para descargar la página de interés (que contiene las cotizaciones de las acciones del IBEX 35 en tiempo _casi_ real).

In [ ]:
base_url = "https://www.eleconomista.es/indice/IBEX-35"
res = requests.get(base_url)
contenido = res.content

La siguiente línea procesa el HTML de la página que hemos descargado:

In [ ]:
soup = BeautifulSoup(contenido, "html.parser")

Una vez procesado el HTML, es posible buscar elementos dentro de él. En particular, podemos buscar los elementos de tipo `table`, es decir, tablas.

In [ ]:
tablas = soup.find_all('table')

El objeto `tablas` contiene todas las tablas presentes en la página. Hay que tener cuidado con dichas tablas porque muchas páginas utilizan elementos de tipo `table` para estructurar el contenido. Por eso, en algunas páginas, aunque parezca haber una única tabla, puede haber otras con una información no interesante que toca descartar.

In [ ]:
len(tablas)

Podemos extraer las filas de todas estas tablas

In [ ]:
lineas = [x for tabla in tablas for x in tabla.find_all('tr')]

para luego extraer los contenidos de cada fila individualmente haciendo

In [ ]:
datos = [[x.text for x  in linea.find_all('td')] for linea in lineas]

Podemos inspeccionar parte del objeto resultante:

In [ ]:
datos[0:3]

Vemos que hay filas que contienen la información de interés junto con otras que contienen cabeceras y otra información irrelevante. En general, la situación puede ser más complicada y se hace necesario estudiar el objeto `tablas` para seleccionar la de interés.

En nuestro caso, podemos filtrar las líneas menos relevantes así:

In [ ]:
datos = [x for x in datos if len(x) > 0]

Finalmente, podemos crear una tabla de Pandas:

In [ ]:
datos = pd.DataFrame(datos)
datos

#### Ejercicio

Usa los elementos `th` de la primera fila de las tablas para extraer nombres para las columnas de la tabla. 

#### Ejercicio

Elimina las columnas irrelevantes y cambia los nombres de las columnas por otros breves y sin caracteres extraños o que dificulten el posproceso.

#### Ejercicio

Cambia el formato de las columnas adecuadamente: convierte a numéricas, etc., las columnas que lo requieran.

## Riesgos del scraping

El scraping es una técnica potente pero tiene varios contras:

* Implica mayor tiempo de desarrollo y mayor esfuerzo en la limpieza de datos (en comparación con otras fuentes como APIs, BDs, ...)
* Si hay que scrapear gran cantidad de páginas, es lento
* Los servidores objetivo de nuestro scraping pueden tener técnicas para evitarlo. Por ejemplo, bloquear la IP temporalmente o introducir delays en las respuestas si hacemos muchas peticiones en poco tiempo. Esto pasa especialmente en las grandes webs recelosas de sus datos (p.e. linkedin, amazon, ...).
* El código de scraping escrito hoy puede no funcionar mañana, si la web destino cambia nombres, etiquetas o estructura. Si se sube a producción para lanzarlo periódicamente, hay que ser conscientes de que en algún momento fallará, y establecer mecanismos de alerta

## Javascript

Es posible que te encuentres con algún caso en el que no puedas descargar tal cual el html y parsearlo, principalmente por dos motivos:

* La estructura de la página se genera parcial o totalmente en cliente
* Debemos interactuar con algún elemento para mostrar la información que queremos (p.e. completar un campo de búsuqeda, hacer click en algún botón, ...)

En estos casos, hay que ejecutar en un navegador local el código javascript de la página destino. Para esta tarea, puedes utilizar [Selenium]().

[Aquí](https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251) un post con un ejemplo de uso.